In [1]:
from requests import get

url = 'https://www.imdb.com/search/title?release_date=2018&sort=num_votes,desc&page=1&ref_=adv_prv'

response = get(url)
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


In [2]:
from bs4 import BeautifulSoup

html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [3]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [53]:
first_movie = movie_containers[0]


In [6]:
first_movie.div

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt4154756"></div>
</div>

In [7]:
first_movie.a

<a href="/title/tt4154756/?ref_=adv_li_i"> <img alt="Avengers: Infinity War" class="loadlate" data-tconst="tt4154756" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BMjMxNjY2MDU1OV5BMl5BanBnXkFtZTgwNzY1MTUwNTM@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB499613450_.png" width="67"/>
</a>

In [13]:
first_name = first_movie.h3.a.text
first_name

'Avengers: Infinity War'

In [18]:
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold')
first_year.text

'(2018)'

In [25]:
first_movie.strong.text

'8.7'

In [26]:
first_imdb = float(first_movie.strong.text)
first_imdb


8.7

In [29]:
first_metascore = first_movie.find('span', class_ = 'metascore favorable')
first_metascore

<span class="metascore favorable">68        </span>

In [30]:
first_mscore = first_movie.find('span', class_ = 'metascore favorable')

first_mscore = int(first_mscore.text)
print(first_mscore)

68


In [33]:
first_votes = first_movie.find('span', attrs = {'name':'nv'})
first_votes['data-value']

'403941'

In [34]:
first_votes = int(first_votes['data-value'])

In [36]:
twelve_movie_mscore = movie_containers[11].find('div', class_ = 'ratings-metascore')
type(twelve_movie_mscore)

NoneType

In [37]:
# Lists to store the scraped data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Extract data from individual movie container
for container in movie_containers:

    # If the movie has Metascore, then extract:
    if container.find('div', class_ = 'ratings-metascore') is not None:

        # The name
        name = container.h3.a.text
        names.append(name)

        # The year
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)

        # The IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        # The Metascore
        m_score = container.find('span', class_ = 'metascore').text
        metascores.append(int(m_score))

        # The number of votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))

In [38]:
import pandas as pd

test_df = pd.DataFrame({'movie': names,
                       'year': years,
                       'imdb': imdb_ratings,
                       'metascore': metascores,
                       'votes': votes})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
movie        36 non-null object
year         36 non-null object
imdb         36 non-null float64
metascore    36 non-null int64
votes        36 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.5+ KB
None


,movie,year,imdb,metascore,votes
0,Avengers: Infinity War,(2018),8.7,68,403941
1,Black Panther,(2018),7.5,88,358365
2,Deadpool 2,(2018),8.1,66,182505
3,Ready Player One,(2018),7.6,64,180195
4,A Quiet Place,(2018),7.8,82,171120
5,Annihilation,(2018),7.0,79,155409
6,Solo: A Star Wars Story,(2018),7.1,62,111276
7,Tomb Raider,(2018),6.4,48,110245
8,Game Night,(I) (2018),7.0,66,97929
9,Jurassic World: Fallen Kingdom,(2018),6.5,51,91987


In [39]:
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2018)]

In [40]:
from time import sleep
from random import randint

In [44]:
from IPython.core.display import clear_output
from time import time

start_time = time()
requests = 0

for _ in range(5):
    # A request would go here
    requests += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.3827359016208247 requests/s


In [45]:
from warnings import warn

warn("Warning Simulation")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Warning Simulation
  This is separate from the ipykernel package so we can avoid doing imports until


In [50]:
headers = {"Accept-Language": "en-US, en;q=0.5"}
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url + 
        '&sort=num_votes,desc&page=' + page, headers = headers)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 72:
            warn('Number of requests was greater than expected.')  
            break 

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request:72; Frequency: 0.07445017441677537 requests/s


In [52]:
movie_ratings = pd.DataFrame({'movie': names,
                              'year': years,
                              'imdb': imdb_ratings,
                              'metascore': metascores,
                              'votes': votes})
print(movie_ratings.info())
movie_ratings.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2949 entries, 0 to 2948
Data columns (total 5 columns):
movie        2949 non-null object
year         2949 non-null object
imdb         2949 non-null float64
metascore    2949 non-null int64
votes        2949 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 115.3+ KB
None


,movie,year,imdb,metascore,votes
0,Gladiator,(2000),8.5,67,1142884
1,Memento,(2000),8.5,80,981330
2,Snatch,(2000),8.3,55,687181
3,Requiem for a Dream,(2000),8.3,68,666682
4,X-Men,(2000),7.4,64,512498
5,Cast Away,(2000),7.8,73,450397
6,American Psycho,(2000),7.6,64,409309
7,Unbreakable,(2000),7.3,62,297828
8,Meet the Parents,(2000),7.0,73,284344
9,Mission: Impossible II,(2000),6.1,59,271688


In [54]:
movie_ratings.to_csv('movie_ratings.csv')